# Proyecto IA - Asistente Inteligente para la Consulta y Resumen de Fichas Técnicas de Medicamentos

Módulo Inteligencia Artificial - Bootcamp BD15 - *KeepCoding*

---

El objetivo de este proyecto es aplicar técnicas de procesamiento de lenguaje natural (NLP) y modelos de lenguaje (LLMs) para extraer, analizar y resumir información médica real contenida en fichas técnicas de medicamentos.

Como fuente de datos se han utilizado fichas técnicas oficiales en formato PDF, convertidas posteriormente a archivos .txt, descargadas manualmente desde el sitio web de la Agencia Española de Medicamentos y Productos Sanitarios (AEMPS).

A lo largo de esta práctica, se han aplicado técnicas como la división del texto en fragmentos (*chunking*), la generación de representaciones vectoriales mediante `OpenAIEmbeddings`, y la construcción de un índice semántico con FAISS para facilitar búsquedas contextuales.

Los pasos clave realizados han sido:

- Conversión de las fichas técnicas en texto plano.
- Extracción de las secciones más relevantes (como indicaciones, posología o efectos adversos).
- División de los textos en fragmentos y generación de **embeddings**.
- Construcción de un índice semántico con **FAISS** para realizar búsquedas por similitud.
- Uso de **prompts personalizados** y un modelo LLM (`ChatOpenAI`) para generar respuestas automáticas.


In [1]:
!pip install openai langchain langchain-community langchain-openai faiss-cpu PyMuPDF tiktoken pypdf


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Importación de librerías
# Librerías estándar
import os
import getpass
import tiktoken

# Lectura de PDFs
import fitz 

# Librerías LangChain y OpenAI
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.prompts import PromptTemplate

# Visualización
from IPython.display import Markdown

# Funciones personalizadas
from utils import convert_pdfs_a_txt

In [3]:
api_key = getpass.getpass("Introduce tu clave de OpenAI:")
os.environ["OPENAI_API_KEY"] = api_key

### 1. Preparación del corpus a partir de fichas técnicas de medicamentos

En esta primera fase del proyecto, se parte de un conjunto de fichas técnicas reales de medicamentos, descargadas manualmente desde el portal oficial de la Agencia Española de Medicamentos y Productos Sanitarios (AEMPS - CIMA).

Estos documentos se encontraban originalmente en formato PDF, y son convertidos a archivos `.txt` mediante la función `convert_pdfs_a_txt()`.

Se han seleccionado medicamentos de uso común, y sus fichas técnicas contienen secciones clínicas clave como:

- Indicaciones terapéuticas  
- Posología y forma de administración  
- Reacciones adversas  
- Interacciones medicamentosas  
- Contraindicaciones y advertencias  

Cada archivo `.txt` representa el contenido completo de una ficha técnica y se encuentra almacenado en la carpeta `./data`.

In [7]:
# Convertimos los archivos PDFs a .txt 
convert_pdfs_a_txt("data")


Convertido: Ibuprofeno.pdf → Ibuprofeno.txt
Convertido: Metformina.pdf → Metformina.txt
Convertido: Lorazepam.pdf → Lorazepam.txt
Convertido: Omeprazol.pdf → Omeprazol.txt


Cargamos todas las fichas técnicas en formato .txt desde la carpeta `data/` y las convertimos en objetos de tipo Document de LangChain, listos para ser procesados.

In [8]:
# Cargar todos los .txt desde la carpeta 'data'
carpeta_data = "data"
archivos_txt = [os.path.join(carpeta_data, f) for f in os.listdir(carpeta_data) if f.endswith(".txt")]

documentos = []
for archivo in archivos_txt:
    loader = TextLoader(archivo, encoding="utf-8")
    docs = loader.load()
    
    for doc in docs:
        doc.metadata["fuente"] = os.path.basename(archivo)
        documentos.append(doc)

print(f"Se cargaron {len(documentos)} fragmentos desde {len(archivos_txt)} archivos.")

Se cargaron 4 fragmentos desde 4 archivos.


## 2. Procesamiento de fichas técnicas y creación de un índice semántico con FAISS

Una vez cargadas las fichas técnicas en formato texto, se procede a su preprocesamiento y vectorización con el objetivo de transformarlas en representaciones numéricas (embeddings) que capturen el significado semántico de su contenido.

Este paso es muy importante para que las consultas médicas puedan generar respuestas específicas de manera contextualizada.

Los pasos realizados en esta sección son:

1. Segmentación del contenido completo de cada ficha técnica en fragmentos más pequeños (*chunking*).

2. Conversión de cada fragmento en un vector numérico mediante el modelo `text-embedding-ada-002` de OpenAI, a través del componente `OpenAIEmbeddings` de LangChain.

3. Construcción de un índice vectorial utilizando **FAISS**, una herramienta que permite realizar búsquedas semánticas eficientes sobre grandes volúmenes de texto.

Este índice semántico será la base para realizar las futuras consultas, generar resúmenes automáticos u obtener respuestas específicas sobre los medicamentos.



In [9]:
# Dividir los documentos en fragmentos más pequeños
splitter = CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)
chunks = splitter.split_documents(documentos)
print(f"Se han generado {len(chunks)} fragmentos de texto.")

Se han generado 278 fragmentos de texto.


In [10]:
# Ejemplo de visualización de un fragmento
print(chunks[0].page_content[:500])

1 de 20 
FICHA TÉCNICA 
1. NOMBRE DEL MEDICAMENTO  
Lorazepam Normogen 2 mg comprimidos EFG 
2. COMPOSICIÓN CUALITATIVA Y CUANTITATIVA  
Cada comprimido contiene 2 mg de lorazepam 
Excipiente(s) con efecto conocido 
Cada comprimido contiene 55,6  mg de lactosa. 
Para consultar la lista completa de excipientes, ver sección 6.1.  
3. FORMA FARMACÉUTICA  
Comprimidos. 
Comprimidos de color amarillo, redondos, biconvexos, serigrafiados con “LZ” en una cara y 
ranurados en la otra, con un diámetro de


In [12]:

# Crear embeddings para cada fragmento
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Construcción del índice
vectorstore = FAISS.from_documents(chunks, embeddings)

# Guardar el índice en local
vectorstore.save_local("faiss_index")

## 3. Consulta y generación de respuestas con un modelo LLM (RAG)

En esta parte de la práctica se lleva a cabo una consulta sobre las fichas técnicas utilizando el índice semántico previamente creado y un modelo de lenguaje (LLM) para generar una respuesta clara y contextualizada.  

Para ello, se ha utilizado la técnica conocida como **Retrieval-Augmented Generation (RAG)**.

Los pasos realizados han sido los siguientes:

1. **Planteamiento de la pregunta:** se redacta una consulta médica relacionada con un medicamento.

2. **Búsqueda semántica con FAISS:** se recuperan los fragmentos de texto más relevantes en función del significado de la pregunta.

3. **Filtrado por medicamento:** solo se conservan los fragmentos que pertenecen al archivo del medicamento consultado.

4. **Construcción del prompt:** se combinan los fragmentos encontrados y la pregunta en un formato estructurado que pueda entender el modelo.

5. **Generación de la respuesta:** el modelo `ChatOpenAI` responde basándose únicamente en la información contextual proporcionada, reduciendo así el riesgo de errores o alucinaciones.

Este proceso simula cómo funcionaría un asistente inteligente capaz de buscar en documentos técnicos y ofrecer respuestas precisas a consultas médicas concretas.

In [14]:
# Realizar la consulta
pregunta = "¿Qué contraindicaciones tiene el Omeprazol?"

# Recuperar fragmentos relevantes desde el índice FAISS
resultados = vectorstore.similarity_search(pregunta, k=10)

# Filtrar fragmentos del medicamento específico
medicamento = "Omeprazol"
resultados_filtrados = [
    r for r in resultados if medicamento.lower() in r.metadata.get("fuente", "").lower()
]
# Vista previa de fragmentos
for i, doc in enumerate(resultados_filtrados, 1):
    print(f"Fragmento {i} (fuente: {doc.metadata.get('fuente')}):")
    print(doc.page_content[:500], "...\n---\n")

# Generar respuesta con LLM
if resultados_filtrados:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
    contexto = "\n".join(r.page_content for r in resultados_filtrados)

    prompt = f"""
Contesta a la siguiente pregunta utilizando únicamente la información del contexto proporcionado.

### Contexto:
{contexto}

### Pregunta:
{pregunta}

### Respuesta:
"""
    respuesta = llm.invoke(prompt)         
    display(Markdown(f"Respuesta generada:\n\n{respuesta.content}"))
else:
    print(f"No se encontraron fragmentos relevantes para el medicamento {medicamento}.")


Fragmento 1 (fuente: Omeprazol.txt):
Durante el tratamiento con omeprazol no se ha observado taquifilaxia.  
 
Efecto sobre H. pylori  
H. pylori está asociado a las úlceras pépticas, incluyendo la úlcera duodenal y gástrica. H. pylori es un 
factor importante para el desarrollo de gastritis. H. pylori junto con la acidez gástrica son los factores 
principales para el desarrollo de úlceras pépticas. H. pylori es un importante factor en el desarrollo de 
gastritis atrófica, que se asocia a un aumento del riesgo de padecer carcinoma  ...
---

Fragmento 2 (fuente: Omeprazol.txt):
Omeprazol es un inhibidor del CYP2C19. Al iniciar o finalizar el tratamiento con omeprazol, debe 
considerarse el potencial de interacciones con medicamentos metabolizados a través del CYP2C19. Se ha 
observado una interacción entre clopidogrel y omeprazol (ver sección 4.5). La relevancia clínica de esta 
interacción no está clara. Como precaución, debería desaconsejarse el uso concomitante de omeprazol y 
clopid

Respuesta generada:

El omeprazol está contraindicado en pacientes que presenten hipersensibilidad al principio activo o a alguno de los excipientes, así como en pacientes con insuficiencia hepática grave. También se desaconseja su uso concomitante con clopidogrel debido a una posible interacción.

### 4. Conclusiones

En esta práctica se ha probado cómo usar modelos de lenguaje (LLMs) para trabajar con información clínica de medicamentos. A partir de fichas técnicas descargadas en formato texto, se ha construido un sistema que permite hacer preguntas médicas y obtener respuestas generadas por un modelo de lenguaje, utilizando únicamente el contenido de los documentos.

Se ha creado un sistema que combina la recuperación semántica de información con la generación de respuestas en lenguaje natural. Para ello, se han utilizado herramientas de LangChain, como la generación de embeddings con OpenAI, la construcción de un índice con FAISS y la integración con el modelo `ChatOpenAI` para responder a preguntas médicas concretas.

La técnica de **Retrieval-Augmented Generation (RAG)**, ha demostrado ser útil para localizar fragmentos relevantes en documentos largos y generar respuestas contextualizadas basadas únicamente en el contenido recuperado.

Durante el desarrollo se han tenido en cuenta aspectos importantes como los límites de tokens, la preparación del corpus en fragmentos adecuados (*chunking*) y la importancia de diseñar prompts claros para obtener respuestas precisas.